In [11]:
#Dumb charging
#!/usr/bin/python
import csv
import numpy as np
from scipy import interpolate
from gurobipy import *

eff = 0.95
Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []

def parameterdc( M, T, Eint,Ereq,TS,TD):

    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
        #print("StartTime:" , TS[i])
        #print("DepartureTime:" , TD[i])
        #print("Initial energy:" , Eint[i])
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/household8_profile.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHP_a = float(row[0])
        HHP_b = float(row[1])
        HHP_c = float(row[2])
        HHQ_a = float(row[3])
        HHQ_b = float(row[4])
        HHQ_c = float(row[5])
        HH.append([HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PV_P = float(row[0])
        PV.append(PV_P)
    m = Model("Generalizeparam")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    
    #Phase constraint:
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
    
    #power outside TS TD as 0:
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
            
    #Subcost calculation:
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            e[i][t] +=  p[i,t] * eff * 1/4  # Energy consumed by EV1   1/4 *
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]    
            te[i] += e[i][t]
            n = te[i]
        initial = IE[i] 
        required = RE 
        I_E = int(np.round(initial))
        R_E = int(np.round(required))
        m.addConstr(n >=0,name="SOC_minconstraint[%d]" %i)
        m.addConstr(n <=24,name="SOC_maxconstraint[%d]" %i)
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
        
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    
    #set objective:
    m.ModelSense = GRB.MAXIMIZE
    m.setObjective(C)
    
    #writing the model:
    m.write('dumbcharging.lp')
    
    # Compute optimal solution:
    m.optimize()
    m.write('dumbcharge.sol')
    #print solution: 

    for v in m.getVars():
        print(v.varName, v.x)
    
    print('Obj:', m.objVal)
    
    with open('dumbcharge.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dumbcharge.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
    return;

#Eint = [30,20];
#Ereq = [80,80];
#TS = [0, 5];       
#TD = [12, 17];
def ev1arrdeptime():
    with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EV1sampledata.csv') as f:
        reader = csv.reader(f)
        header = next(reader)
        y = []
        for row in reader :
            ydist = float(row[0])
            yaDate = row[1]
            ydDate = row[2]
            y.append([ydist,yaDate,ydDate])
        ev1 =(random.choice(y))
    return(ev1);
ev_a=[0 for i in range(M)]
ev_d=[0 for i in range(M)]
Ev = [0 for i in range(M)]
IE = [0 for i in range(M)]
RE = 17.0
Ev[0] = ev1arrdeptime()
Ev[1] = ev1arrdeptime()
Date = '01.01.2014 '
Time = ':00'
Starttime =[0 for i in range(M)]
Departtime =[0 for i in range(M)]
for i in range(0,M):
    Starttime[i] = Date + Ev[i][1] +Time
    Departtime[i] = Date + Ev[i][2] +Time
    print('energy:',Ev[i][0])
    IE[i] = RE - Ev[i][0]
    print('Initial energy:',IE[i])
    print('ev_a:',Starttime[i])
    print('ev_d:',Departtime[i])
    for item,val in thisdict.items():
        if item == Starttime[i]:
            ev_a[i] = val
        elif item == Departtime[i]:
            ev_d[i] = val
if ev_a[0] < ev_a[1]:        
    TS = [ev_a[0],ev_a[1]]  
    TD= [ev_d[0],ev_d[1]] 
    print('ev1 arrives before')
    print(TS)
    print(TD)
else:
    TS = [ev_a[1],ev_a[0]]   
    TD= [ev_d[1],ev_d[0]]
    print('ev2 arrives before')
    print(TS)
    print(TD)
parameterdc(2,96,IE,RE,TS,TD)
    

# do IIS
#    print('The model is infeasible; computing IIS')
#    m.computeIIS()
#    if m.IISMinimal:
#      print('IIS is minimal\n')
#    else:
#      print('IIS is not minimal\n')
#    print('\nThe following constraint(s) cannot be satisfied:')
#    for c in m.getConstrs():
#        if c.IISConstr:
#             print('%s' % c.constrName)

energy: 7.48
Initial energy: 9.52
ev_a: 01.01.2014 13:00:00
ev_d: 01.01.2014 16:00:00
energy: 7.48
Initial energy: 9.52
ev_a: 01.01.2014 18:45:00
ev_d: 01.01.2014 23:00:00
ev1 arrives before
[52, 75]
[64, 92]
Optimize a model with 357 rows, 768 columns and 768 nonzeros
Variable types: 192 continuous, 576 integer (576 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [7e-02, 8e-02]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 4.4337450
Presolve removed 357 rows and 768 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 16 available processors)

Solution count 2: 4.50148 4.43374 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.501478807000e+00, best bound 4.501478807000e+00, gap 0.0000%
a[0,0] 1.0
a[0,1] 1.0
a[0,2] 1.0
a[0,3] 1.0
a[0,4] 1.0
a[0,5] 1.0
a[0,6] 1.0
a[0,7] 1.0
a[0,8] 1.0
a[

In [ ]:
#dayahead scheduling
#!/usr/bin/python
import csv
import numpy as np
from datetime import time,datetime,date
from scipy import interpolate
from gurobipy import *
import random

Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []
thisdict = {
  '01.01.2014 00:00:00': 0,
  '01.01.2014 00:15:00': 1,
  '01.01.2014 00:30:00': 2
}
thisdict.update([ ('01.01.2014 00:45:00', 3) , ('01.01.2014 01:00:00', 4) , ('01.01.2014 01:15:00', 5) ,
                 ('01.01.2014 01:30:00', 6),('01.01.2014 01:45:00', 7),('01.01.2014 02:00:00', 8),
                 ('01.01.2014 02:15:00', 9),('01.01.2014 02:30:00', 10),('01.01.2014 02:45:00', 11),
                 ('01.01.2014 03:00:00', 12),('01.01.2014 03:15:00', 13),('01.01.2014 03:30:00', 14) , 
                 ('01.01.2014 03:45:00', 15) , ('01.01.2014 04:00:00', 16) , ('01.01.2014 04:15:00', 17),
                 ('01.01.2014 04:30:00', 18),('01.01.2014 04:45:00', 19),('01.01.2014 05:00:00', 20),
                 ('01.01.2014 05:15:00', 21),('01.01.2014 05:30:00', 22),('01.01.2014 05:45:00', 23),
                 ('01.01.2014 06:00:00', 24),('01.01.2014 06:15:00', 25) , ('01.01.2014 06:30:00', 26),
                 ('01.01.2014 06:45:00', 27),('01.01.2014 07:00:00', 28),('01.01.2014 07:15:00', 29),
                 ('01.01.2014 07:30:00', 30),('01.01.2014 07:45:00', 31),('01.01.2014 08:00:00', 32),
                 ('01.01.2014 08:15:00', 33),('01.01.2014 08:30:00', 34) , ('01.01.2014 08:45:00', 35) , 
                 ('01.01.2014 09:00:00', 36) , ('01.01.2014 09:15:00', 37),('01.01.2014 09:30:00', 38),
                 ('01.01.2014 09:45:00', 39),('01.01.2014 10:00:00', 40),('01.01.2014 10:15:00', 41),
                 ('01.01.2014 10:30:00', 42),('01.01.2014 10:45:00', 43),('01.01.2014 11:00:00', 44),
                 ('01.01.2014 11:15:00', 45) , ('01.01.2014 11:30:00', 46),('01.01.2014 11:45:00', 47),
                 ('01.01.2014 12:00:00', 48),('01.01.2014 12:15:00', 49),('01.01.2014 12:30:00', 50),
                 ('01.01.2014 12:45:00', 51),('01.01.2014 13:00:00', 52),('01.01.2014 13:15:00', 53),
                 ('01.01.2014 13:30:00', 54) , ('01.01.2014 13:45:00', 55) , ('01.01.2014 14:00:00', 56) , 
                 ('01.01.2014 14:15:00', 57),('01.01.2014 14:30:00', 58),('01.01.2014 14:45:00', 59),
                 ('01.01.2014 15:00:00', 60),('01.01.2014 15:15:00', 61),('01.01.2014 15:30:00', 62),
                 ('01.01.2014 15:45:00', 63),('01.01.2014 16:00:00', 64),('01.01.2014 16:15:00', 65) , 
                 ('01.01.2014 16:30:00', 66),('01.01.2014 16:45:00', 67),('01.01.2014 17:00:00', 68),
                 ('01.01.2014 17:15:00', 69),('01.01.2014 17:30:00', 70),('01.01.2014 17:45:00', 71),
                 ('01.01.2014 18:00:00', 72),('01.01.2014 18:15:00', 73),('01.01.2014 18:30:00', 74) , 
                 ('01.01.2014 18:45:00', 75) , ('01.01.2014 19:00:00', 76) , ('01.01.2014 19:15:00', 77),
                 ('01.01.2014 19:30:00', 78),('01.01.2014 19:45:00', 79),('01.01.2014 20:00:00', 80),
                 ('01.01.2014 20:15:00', 81),('01.01.2014 20:30:00', 82),('01.01.2014 20:45:00', 83),
                 ('01.01.2014 21:00:00',84),('01.01.2014 21:15:00', 85) , ('01.01.2014 21:30:00', 86),
                 ('01.01.2014 21:45:00', 87),('01.01.2014 22:00:00', 88),('01.01.2014 22:15:00', 89),
                 ('01.01.2014 22:30:00', 90),('01.01.2014 22:45:00', 91),('01.01.2014 23:00:00', 92),
                 ('01.01.2014 23:15:00', 93),('01.01.2014 23:30:00', 94) , ('01.01.2014 23:45:00', 95)])
def parameterda( M, T, IE,RE,TS,TD):

    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/HH_profile_Scheduling.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHDate = row[0]
        HHP_a = float(row[1])
        HHP_b = float(row[2])
        HHP_c = float(row[3])
        HHQ_a = float(row[4])
        HHQ_b = float(row[5])
        HHQ_c = float(row[6])
        HH.append([HHDate,HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile_Scheduling.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PVDate = row[0]
        PV_P = float(row[1])
        PV.append([PVDate,PV_P])
    m = Model("dayahead")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    #Phase constraint
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
            
    #SOC user constraint
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            if p[i,t] >= 0:                     # if power val is +, charging so eff is 0.9
                e[i][t] +=  p[i,t] * 0.9 * 1/4  # energy consumed
            else:                               # if power val is -, charging so eff is 1.1
                e[i][t] +=  p[i,t] * 1.1 * 1/4  
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]
            te[i] += e[i][t]
            n = te[i]
        initial = IE[i] 
        required = RE 
        I_E = int(np.round(initial))
        R_E = int(np.round(required))
        m.addConstr(n >=0,name="SOC_minconstraint[%d]" %i)
        m.addConstr(n <=24,name="SOC_maxconstraint[%d]" %i)
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
    
    #power outside TS TD as 0
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))

    #Grid constraint
    for t in range(STime[i],DTime[i]):
        for i in range(M):
            pa[i][t] = a[i,t] * p[i,t]
            pb[i][t] = b[i,t] * p[i,t]
            pc[i][t] = c[i,t] * p[i,t]
            
            patot[t] += pa[i][t]
            pbtot[t] += pb[i][t]
            pctot[t] += pc[i][t]

        pahhpv[t] = patot[t] + HH[t][1] - PV[t][1]
        pbhhpv[t] = pbtot[t] + HH[t][2]
        pchhpv[t] = pctot[t] + HH[t][3]
    
        pab[t] = pahhpv[t] - pbhhpv[t]
        pbc[t] = pbhhpv[t] - pchhpv[t]
        pca[t] = pchhpv[t] - pahhpv[t]
    
        m.addConstr(pab[t] <= 4.60, name="Power_A_B_maxconstraint")
        m.addConstr(pab[t] >= -4.60, name="Power_A_B_minconstraint")
        m.addConstr(pbc[t] <= 4.60, name="Power_B_C_maxconstraint")
        m.addConstr(pbc[t] >= -4.60, name="Power_B_C_minconstraint")
        m.addConstr(pca[t] <= 4.60, name="Power_C_A_maxconstraint")
        m.addConstr(pca[t] >= -4.60, name="Power_C_A_minconstraint")
    
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    Im_cost = C
    ST = STime[0]
    i = M - 1
    DT = DTime[i]
    for t in range(ST,DT):
        if pab[t] <= 2.3:
            Im_cost += 0.0
        elif pab[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pbc[t] <= 2.3:
            Im_cost += 0.0
        elif pbc[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pca[t] <= 2.3:
            Im_cost += 0.0
        elif pca[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
    
    #set objective:
    m.ModelSense = GRB.MINIMIZE
    Im_C = Im_cost
    #m.setObjective(C)
    m.setObjective(Im_C)

    m.write('dayahead.lp')

    # Compute optimal solution
    m.optimize()
    m.write('dayahead.sol')
    
    #print solution 
    for v in m.getVars():
        print(v.varName, v.x)
    
    print('Obj:', m.objVal)
    finalcost = m.objVal

# do IIS
#    print('The model is infeasible; computing IIS')
#    m.computeIIS()
#    if m.IISMinimal:
#      print('IIS is minimal\n')
#    else:
#      print('IIS is not minimal\n')
#    print('\nThe following constraint(s) cannot be satisfied:')
#    for c in m.getConstrs():
#        if c.IISConstr:
#             print('%s' % c.constrName)
    return(finalcost);
def ev1arrdeptime():
    with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv') as f:
        reader = csv.reader(f)
        header = next(reader)
        y = []
        for row in reader :
            ydist = float(row[0])
            yaDate = row[1]
            ydDate = row[2]
            y.append([ydist,yaDate,ydDate])
        ev1 =(random.choice(y))
    return(ev1);
M = 2
T = 96
InitialEgy = [0 for i in range(M)]
IEgy = [0 for i in range(M)]
ev_a=[0 for i in range(M)]
ev_d=[0 for i in range(M)]
Ev = [0 for i in range(M)]
IE = [0 for i in range(M)]
RE = 17.0
Ev[0] = ev1arrdeptime()
Ev[1] = ev1arrdeptime()
Date = '01.01.2014 '
Time = ':00'
Starttime =[0 for i in range(M)]
Departtime =[0 for i in range(M)]
for i in range(0,M):
    Starttime[i] = Date + Ev[i][1] +Time
    Departtime[i] = Date + Ev[i][2] +Time
    print('energy:',Ev[i][0])
    IE[i] = RE - Ev[i][0]
    print('Initial energy:',IE[i])
    print('ev_a:',Starttime[i])
    print('ev_d:',Departtime[i])
    for item,val in thisdict.items():
        if item == Starttime[i]:
            ev_a[i] = val
        elif item == Departtime[i]:
            ev_d[i] = val
if ev_a[0] < ev_a[1]:        
    TS = [ev_a[0],ev_a[1]]  
    TD= [ev_d[0],ev_d[1]] 
    IE =[IE[0],IE[1]]
    print('ev1 arrives before')
    print(TS)
    print(TD)
else:
    TS = [ev_a[1],ev_a[0]]   
    TD= [ev_d[1],ev_d[0]]
    IE =[IE[1],IE[0]]
    print('ev2 arrives before')
    print(TS)
    print(TD)

if TD[0] > TS[1] and TD[0] < 96 and TD[1] > TS[1] and TD[1] < 96:
    if TD[0] < TD[1] or TD[1] < TD[0] or TD[1] == TD[0]:
        print('case 1: Both EVs depart before mid-night')
        #Start = [TS[0],TS[1]]
        #Depart = [TD[0],TD[1]]
        #cost1=parameterda(2,96,IE,RE,Start,Depart)
        #with open('dayahead.sol', newline='') as f:
        #    reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        #    next(reader)
        #    next(reader)
        #    lines = list(reader)

        #with open('dayahead.csv', 'w', newline='') as myfile:
        #    wr = csv.writer(myfile, delimiter=',')
        #    wr.writerows(lines)
    
        #f.close()
        #myfile.close()      
        #finalcost = cost1
        #print('final optimal cost:',finalcost)
    
elif TD[0] < TS[0] and TD[0] >= 0 and TD[1] < TS[0] and TD[1] >= 0:
    if TD[0] < TD[1] or TD[1] < TD[0]:
        print('case2: Both EVs depart in the morning')
        Start = [TS[0],TS[1]]
        Depart = [95,95]
        cost1=parameterda(2,96,IE,RE,Start,Depart)
        with open('dayahead.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('dayaheadcase2.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
    
        with open('dayaheadcase2.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            power =[]
            p = []
            for row in csv_reader:
                power_str = row[0]
                power_P = float(row[1])
                power.append([power_str,power_P])
            for i in range(len(power)):
                for j in range(TS[0],96):
                    if power[i][0] == 'Power[%d,%d]' %(0,j):
                        p.append(power[i][1])
            print(p)
            energy =[]
            pwr=[0 for i in range(T)]
            for i in range(len(p)):
                if p[i] >= 0:
                    pwr[i] =  p[i] * 0.9 * 1/4  # Energy consumed by EV1   1/4 *
                else:
                    pwr[i] =  p[i] * 1.1 * 1/4  
                energy.append(pwr[i])
                InitialEgy[0] += energy[i]
            print(energy)
            print(InitialEgy[0])
            powr =[]
            po = []
            for row in csv_reader:
                powr_str = row[0]
                powr_P = float(row[1])
                powr.append([powr_str,powr_P])
            for i in range(len(powr)):
                for j in range(TS[1],96):
                    if powr[i][0] == 'Power[%d,%d]' %(1,j):
                        po.append(powr[i][1])
            print(po)
            egy =[]
            pwrs=[0 for i in range(T)]
            for i in range(len(po)):
                if po[i] >= 0:
                    pwrs[i] =  po[i] * 0.9 * 1/4  # Energy consumed by EV1   1/4 *
                else:
                    pwrs[i] =  po[i] * 1.1 * 1/4 
                egy.append(pwrs[i])
                IEgy[1] += egy[i]
            print(egy)
            print(IEgy[1])
        Start = [0,0]
        Depart = [TD[0],TD[1]]
        IE[0] += InitialEgy[0]
        IE[1] += IEgy[1]
        cost2=parameterda(2,96,IE,RE,Start,Depart)
        with open('dayahead.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('dayaheadcase2.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
            
        f.close()
        myfile.close()
        finalcost = cost1 + cost2 
        print('final optimal cost:',finalcost)
elif TD[0] > TS[1] and TD[0] < 96 and TD[1] < TS[0] and TD[1] >= 0:
    print('case3: EV1 depart before mid-night and EV2 departs in the morning')
    Start = [TS[0],TS[1]]
    Depart = [TD[0],95]
    cost1=parameterda(2,96,IE,RE,Start,Depart)
    with open('dayahead.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dayaheadcase3.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    with open('dayaheadcase3.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        powr =[]
        po = []
        for row in csv_reader:
            powr_str = row[0]
            powr_P = float(row[1])
            powr.append([powr_str,powr_P])
        for i in range(len(powr)):
            for j in range(TS[1],96):
                if powr[i][0] == 'Power[%d,%d]' %(1,j):
                    po.append(powr[i][1])
        print(po)
        egy =[]
        pwrs=[0 for i in range(T)]
        for i in range(len(po)):
            if po[i] >= 0:
                pwrs[i] =  po[i] * 0.9 * 1/4  # Energy consumed by EV1   1/4 *
            else:
                pwrs[i] =  po[i] * 1.1 * 1/4 
            egy.append(pwrs[i])
            IEgy[1] += egy[i]
        print(egy)
        print(IEgy[1])
    Start = [0]
    Depart = [TD[1]]
    IE[1] += IEgy[1]
    cost2=parameterda(1,96,IE,RE,Start,Depart)
    with open('dayahead.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dayaheadcase3.csv', 'a', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
            
    f.close()
    myfile.close()
    finalcost = cost1 + cost2 
    print('final optimal cost:',finalcost)      
elif TD[1] > TS[1] and TD[1] < 96 and TD[0] < TS[0] and TD[0] >= 0:
    print('case4: EV2 depart before mid-night and EV1 departs in the morning')
    Start = [TS[0],TS[1]]
    Depart = [95,TD[1]]
    cost1=parameterda(2,96,IE,RE,Start,Depart)
    with open('dayahead.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dayaheadcase4.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    with open('dayaheadcase4.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        power =[]
        p = []
        for row in csv_reader:
            power_str = row[0]
            power_P = float(row[1])
            power.append([power_str,power_P])
        for i in range(len(power)):
            for j in range(TS[0],96):
                if power[i][0] == 'Power[%d,%d]' %(0,j):
                    p.append(power[i][1])
        print(p)
        energy =[]
        pwr=[0 for i in range(T)]
        for i in range(len(p)):
            if p[i] >= 0:
                pwr[i] =  p[i] * 0.9 * 1/4  # Energy consumed by EV1   1/4 *
            else:
                pwr[i] =  p[i] * 1.1 * 1/4  
            energy.append(pwr[i])
            InitialEgy[0] += energy[i]
        print(energy)
        print(InitialEgy[0])
    Start = [0]
    Depart = [TD[0]]
    IE[0] += InitialEgy[0]
    cost2=parameterda(1,96,IE,RE,Start,Depart)
    with open('dayahead.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dayaheadcase4.csv', 'a', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
            
    f.close()
    myfile.close()
    finalcost = cost1 + cost2  
    print('final optimal cost:',finalcost) 
else:
    exit

energy: 3.91
Initial energy: 13.09
ev_a: 01.01.2014 18:30:00
ev_d: 01.01.2014 22:15:00
energy: 7.31
Initial energy: 9.690000000000001
ev_a: 01.01.2014 18:00:00
ev_d: 01.01.2014 10:30:00
ev2 arrives before
[72, 74]
[42, 89]
case4: EV2 depart before mid-night and EV1 departs in the morning
Academic license - for non-commercial use only
Optimize a model with 352 rows, 768 columns and 844 nonzeros
Model has 90 quadratic constraints
Variable types: 192 continuous, 576 integer (576 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [7e-02, 7e-02]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 2e+01]
  QRHS range       [4e+00, 5e+00]
Presolve removed 320 rows and 645 columns
Presolve time: 0.01s
Presolved: 662 rows, 483 columns, 2103 nonzeros
Variable types: 393 continuous, 90 integer (90 binary)

Root relaxation: objective 3.135598e+00, 248 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objec

 3239369 1614403    3.18367   47   35    3.19294    3.17724  0.49%  10.7  310s
 3291391 1637919    3.19192   60   22    3.19294    3.17727  0.49%  10.7  315s
 3343262 1660936     cutoff   54         3.19294    3.17729  0.49%  10.7  320s
 3379413 1676932     cutoff   54         3.19294    3.17731  0.49%  10.7  325s
 3424309 1696769    3.18059   46   40    3.19294    3.17733  0.49%  10.7  330s
 3470472 1717374     cutoff   58         3.19294    3.17736  0.49%  10.7  335s
 3514793 1737214    3.18203   51   34    3.19294    3.17738  0.49%  10.7  340s
 3558883 1757005 infeasible   56         3.19294    3.17740  0.49%  10.7  345s
 3602049 1776248    3.19066   56   23    3.19294    3.17742  0.49%  10.7  350s
 3650780 1798314     cutoff   58         3.19294    3.17744  0.49%  10.7  355s
 3696512 1818726    3.18730   53   30    3.19294    3.17746  0.48%  10.7  360s
 3744926 1840029    3.19029   53   25    3.19294    3.17749  0.48%  10.7  365s
 3792729 1861580     cutoff   51         3.19294    

 7283409 3343189    3.19027   51   30    3.19294    3.17869  0.45%  10.8  830s
 7318719 3357554    3.18725   50   21    3.19294    3.17870  0.45%  10.8  835s
 7357462 3373230    3.18823   56   24    3.19294    3.17871  0.45%  10.8  840s
 7390254 3386529    3.17873   49   37    3.19294    3.17872  0.45%  10.8  845s
 7420902 3399063    3.17877   47   38    3.19294    3.17872  0.45%  10.8  850s
 7459802 3414637    3.18517   51   27    3.19294    3.17873  0.44%  10.8  855s
 7496347 3429179    3.19057   56   24    3.19294    3.17874  0.44%  10.8  860s
 7532971 3444102    3.19272   60   12    3.19294    3.17875  0.44%  10.8  865s
 7571499 3459452    3.18358   53   28    3.19294    3.17876  0.44%  10.8  870s
 7608002 3473756    3.18719   46   28    3.19294    3.17877  0.44%  10.8  875s
 7646663 3489443    3.19232   57   24    3.19294    3.17878  0.44%  10.8  880s
 7679652 3502612    3.18699   53   29    3.19294    3.17879  0.44%  10.8  885s
 7719068 3518548    3.18812   52   25    3.19294    

 11171404 4848989    3.19132   54   22    3.19294    3.17952  0.42%  10.8 1350s
 11206173 4861751    3.18784   50   31    3.19294    3.17953  0.42%  10.8 1355s
 11238864 4874034    3.18658   50   25    3.19294    3.17953  0.42%  10.8 1360s
 11273585 4886628    3.18872   56   28    3.19294    3.17954  0.42%  10.8 1365s
 11307675 4899180    3.18784   49   31    3.19294    3.17955  0.42%  10.8 1370s
 11344315 4912685     cutoff   58         3.19294    3.17955  0.42%  10.8 1375s
 11378858 4925139     cutoff   60         3.19294    3.17956  0.42%  10.8 1380s


In [113]:
#Test dayahead scheduling
#!/usr/bin/python
import csv
import numpy as np
from datetime import time,datetime,date
from scipy import interpolate
from gurobipy import *

eff = 0.95
Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []
thisdict = {
  '01.01.2014 00:00:00': 0,
  '01.01.2014 00:15:00': 1,
  '01.01.2014 00:30:00': 2
}
thisdict.update([ ('01.01.2014 00:45:00', 3) , ('01.01.2014 01:00:00', 4) , ('01.01.2014 01:15:00', 5) ,
                 ('01.01.2014 01:30:00', 6),('01.01.2014 01:45:00', 7),('01.01.2014 02:00:00', 8),
                 ('01.01.2014 02:15:00', 9),('01.01.2014 02:30:00', 10),('01.01.2014 02:45:00', 11),
                 ('01.01.2014 03:00:00', 12),('01.01.2014 03:15:00', 13),('01.01.2014 03:30:00', 14) , 
                 ('01.01.2014 03:45:00', 15) , ('01.01.2014 04:00:00', 16) , ('01.01.2014 04:15:00', 17),
                 ('01.01.2014 04:30:00', 18),('01.01.2014 04:45:00', 19),('01.01.2014 05:00:00', 20),
                 ('01.01.2014 05:15:00', 21),('01.01.2014 05:30:00', 22),('01.01.2014 05:45:00', 23),
                 ('01.01.2014 06:00:00', 24),('01.01.2014 06:15:00', 25) , ('01.01.2014 06:30:00', 26),
                 ('01.01.2014 06:45:00', 27),('01.01.2014 07:00:00', 28),('01.01.2014 07:15:00', 29),
                 ('01.01.2014 07:30:00', 30),('01.01.2014 07:45:00', 31),('01.01.2014 08:00:00', 32),
                 ('01.01.2014 08:15:00', 33),('01.01.2014 08:30:00', 34) , ('01.01.2014 08:45:00', 35) , 
                 ('01.01.2014 09:00:00', 36) , ('01.01.2014 09:15:00', 37),('01.01.2014 09:30:00', 38),
                 ('01.01.2014 09:45:00', 39),('01.01.2014 10:00:00', 40),('01.01.2014 10:15:00', 41),
                 ('01.01.2014 10:30:00', 42),('01.01.2014 10:45:00', 43),('01.01.2014 11:00:00', 44),
                 ('01.01.2014 11:15:00', 45) , ('01.01.2014 11:30:00', 46),('01.01.2014 11:45:00', 47),
                 ('01.01.2014 12:00:00', 48),('01.01.2014 12:15:00', 49),('01.01.2014 12:30:00', 50),
                 ('01.01.2014 12:45:00', 51),('01.01.2014 13:00:00', 52),('01.01.2014 13:15:00', 53),
                 ('01.01.2014 13:30:00', 54) , ('01.01.2014 13:45:00', 55) , ('01.01.2014 14:00:00', 56) , 
                 ('01.01.2014 14:15:00', 57),('01.01.2014 14:30:00', 58),('01.01.2014 14:45:00', 59),
                 ('01.01.2014 15:00:00', 60),('01.01.2014 15:15:00', 61),('01.01.2014 15:30:00', 62),
                 ('01.01.2014 15:45:00', 63),('01.01.2014 16:00:00', 64),('01.01.2014 16:15:00', 65) , 
                 ('01.01.2014 16:30:00', 66),('01.01.2014 16:45:00', 67),('01.01.2014 17:00:00', 68),
                 ('01.01.2014 17:15:00', 69),('01.01.2014 17:30:00', 70),('01.01.2014 17:45:00', 71),
                 ('01.01.2014 18:00:00', 72),('01.01.2014 18:15:00', 73),('01.01.2014 18:30:00', 74) , 
                 ('01.01.2014 18:45:00', 75) , ('01.01.2014 19:00:00', 76) , ('01.01.2014 19:15:00', 77),
                 ('01.01.2014 19:30:00', 78),('01.01.2014 19:45:00', 79),('01.01.2014 20:00:00', 80),
                 ('01.01.2014 20:15:00', 81),('01.01.2014 20:30:00', 82),('01.01.2014 20:45:00', 83),
                 ('01.01.2014 21:00:00',84),('01.01.2014 21:15:00', 85) , ('01.01.2014 21:30:00', 86),
                 ('01.01.2014 21:45:00', 87),('01.01.2014 22:00:00', 88),('01.01.2014 22:15:00', 89),
                 ('01.01.2014 22:30:00', 90),('01.01.2014 22:45:00', 91),('01.01.2014 23:00:00', 92),
                 ('01.01.2014 23:15:00', 93),('01.01.2014 23:30:00', 94) , ('01.01.2014 23:45:00', 95)])
def parameterda( M, T, Eint,Ereq,TS,TD):

    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/HH_profile_Scheduling.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHDate = row[0]
        HHP_a = float(row[1])
        HHP_b = float(row[2])
        HHP_c = float(row[3])
        HHQ_a = float(row[4])
        HHQ_b = float(row[5])
        HHQ_c = float(row[6])
        HH.append([HHDate,HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile_Scheduling.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PVDate = row[0]
        PV_P = float(row[1])
        PV.append([PVDate,PV_P])
    m = Model("dayahead")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    #Phase constraint
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
            
    #SOC user constraint
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            e[i][t] +=  p[i,t] * eff * 1/4  # Energy consumed by EV1   1/4 *
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]
            te[i] += e[i][t]
            n = te[i]
        initial = IE[i] 
        required = RE 
        I_E = int(np.round(initial))
        R_E = int(np.round(required))
        m.addConstr(n >=0,name="SOC_minconstraint[%d]" %i)
        m.addConstr(n <=24,name="SOC_maxconstraint[%d]" %i)
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
    
    #power outside TS TD as 0
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))

    #Grid constraint
    for t in range(T):
        #if HH[i][0] == atime:
        for i in range(M):
    #if i < M :
            pa[i][t] = a[i,t] * p[i,t]
            pb[i][t] = b[i,t] * p[i,t]
            pc[i][t] = c[i,t] * p[i,t]
            
            patot[t] += pa[i][t]
            pbtot[t] += pb[i][t]
            pctot[t] += pc[i][t]
        #print(pa[i][t])
        pahhpv[t] = patot[t] + HH[t][1] - PV[t][1]
        pbhhpv[t] = pbtot[t] + HH[t][2]
        pchhpv[t] = pctot[t] + HH[t][3]
   
    
        pab[t] = pahhpv[t] - pbhhpv[t]
        pbc[t] = pbhhpv[t] - pchhpv[t]
        pca[t] = pchhpv[t] - pahhpv[t]
    
        m.addConstr(pab[t] <= 4.60, name="Power_A_B_maxconstraint")
        m.addConstr(pab[t] >= -4.60, name="Power_A_B_minconstraint")
        m.addConstr(pbc[t] <= 4.60, name="Power_B_C_maxconstraint")
        m.addConstr(pbc[t] >= -4.60, name="Power_B_C_minconstraint")
        m.addConstr(pca[t] <= 4.60, name="Power_C_A_maxconstraint")
        m.addConstr(pca[t] >= -4.60, name="Power_C_A_minconstraint")
    
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    Im_cost = C
    ST = STime[0]
    i = M - 1
    DT = DTime[i]
    for t in range(ST,DT):
        if pab[t] <= 2.3:
            Im_cost += 0.0
        elif pab[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pbc[t] <= 2.3:
            Im_cost += 0.0
        elif pbc[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pca[t] <= 2.3:
            Im_cost += 0.0
        elif pca[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
    
    #set objective:
    m.ModelSense = GRB.MINIMIZE
    Im_C = Im_cost
    #m.setObjective(C)
    m.setObjective(Im_C)

    m.write('dayahead.lp')
    #m.write('testcost.lp')

    # Compute optimal solution
    m.optimize()
    m.write('dayahead.sol')
    #print solution 

    for v in m.getVars():
        print(v.varName, v.x)
    
    print('Obj:', m.objVal)
    
    with open('dayahead.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('dayahead.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
# do IIS
#    print('The model is infeasible; computing IIS')
#    m.computeIIS()
#    if m.IISMinimal:
#      print('IIS is minimal\n')
#    else:
#      print('IIS is not minimal\n')
#    print('\nThe following constraint(s) cannot be satisfied:')
#    for c in m.getConstrs():
#        if c.IISConstr:
#             print('%s' % c.constrName)
    return;
M=2
#Eint = [30,20];
#Ereq = [80,80];
atime =[0 for t in range(M)]
dtime =[0 for t in range(M)]
IE = [0 for i in range(M)]
EV1_a = datetime(2014, 1, 1, 12, 45)
atime[0] = EV1_a.strftime("%d.%m.%Y %H:%M:%S")
EV2_a = datetime(2014, 1, 1, 17, 15)
atime[1] = EV2_a.strftime("%d.%m.%Y %H:%M:%S")
EV1_d = datetime(2014, 1, 1, 19, 45)
dtime[0] = EV1_d.strftime("%d.%m.%Y %H:%M:%S")
EV2_d = datetime(2014, 1, 1, 23, 00)
dtime[1] = EV2_d.strftime("%d.%m.%Y %H:%M:%S")
for i in range(0,M):
    for item,val in thisdict.items():
        if item == atime[i]:
            ev_a[i] = val
        elif item == dtime[i]:
            ev_d[i] = val
TS = [ev_a[0],ev_a[1]]
TD = [ev_d[0],ev_d[1]]
RE = 17.0
IE =[3.4,7.65]
parameterda(2,96,IE,RE,TS,TD)

Optimize a model with 335 rows, 768 columns and 768 nonzeros
Model has 576 quadratic constraints
Variable types: 192 continuous, 576 integer (576 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [7e-02, 8e-02]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 1e+01]
  QRHS range       [3e-01, 9e+00]
Presolve removed 282 rows and 564 columns
Presolve time: 0.04s
Presolved: 1217 rows, 816 columns, 3570 nonzeros
Variable types: 663 continuous, 153 integer (153 binary)

Root relaxation: objective 6.731456e+00, 447 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.73146    0   89          -    6.73146      -     -    0s
H    0     0                       6.9620814    6.73146  3.31%     -    0s
H    0     0                       6.8567922    6.73146  1.83%     -    0s
H

In [ ]:
#Test Dynamic Scheduling
#!/usr/bin/python
import csv
import numpy as np
from scipy import interpolate
from gurobipy import *
import random
import time
import sys

M=2
Eint = []
Ereq = []
Im_C = []
C = []
e = []
TS = []
TD = []
thisdict = {
  '01.01.2014 00:00:00': 0,
  '01.01.2014 00:15:00': 1,
  '01.01.2014 00:30:00': 2
}
thisdict.update([ ('01.01.2014 00:45:00', 3) , ('01.01.2014 01:00:00', 4) , ('01.01.2014 01:15:00', 5) ,
                 ('01.01.2014 01:30:00', 6),('01.01.2014 01:45:00', 7),('01.01.2014 02:00:00', 8),
                 ('01.01.2014 02:15:00', 9),('01.01.2014 02:30:00', 10),('01.01.2014 02:45:00', 11),
                 ('01.01.2014 03:00:00', 12),('01.01.2014 03:15:00', 13),('01.01.2014 03:30:00', 14) , 
                 ('01.01.2014 03:45:00', 15) , ('01.01.2014 04:00:00', 16) , ('01.01.2014 04:15:00', 17),
                 ('01.01.2014 04:30:00', 18),('01.01.2014 04:45:00', 19),('01.01.2014 05:00:00', 20),
                 ('01.01.2014 05:15:00', 21),('01.01.2014 05:30:00', 22),('01.01.2014 05:45:00', 23),
                 ('01.01.2014 06:00:00', 24),('01.01.2014 06:15:00', 25) , ('01.01.2014 06:30:00', 26),
                 ('01.01.2014 06:45:00', 27),('01.01.2014 07:00:00', 28),('01.01.2014 07:15:00', 29),
                 ('01.01.2014 07:30:00', 30),('01.01.2014 07:45:00', 31),('01.01.2014 08:00:00', 32),
                 ('01.01.2014 08:15:00', 33),('01.01.2014 08:30:00', 34) , ('01.01.2014 08:45:00', 35) , 
                 ('01.01.2014 09:00:00', 36) , ('01.01.2014 09:15:00', 37),('01.01.2014 09:30:00', 38),
                 ('01.01.2014 09:45:00', 39),('01.01.2014 10:00:00', 40),('01.01.2014 10:15:00', 41),
                 ('01.01.2014 10:30:00', 42),('01.01.2014 10:45:00', 43),('01.01.2014 11:00:00', 44),
                 ('01.01.2014 11:15:00', 45) , ('01.01.2014 11:30:00', 46),('01.01.2014 11:45:00', 47),
                 ('01.01.2014 12:00:00', 48),('01.01.2014 12:15:00', 49),('01.01.2014 12:30:00', 50),
                 ('01.01.2014 12:45:00', 51),('01.01.2014 13:00:00', 52),('01.01.2014 13:15:00', 53),
                 ('01.01.2014 13:30:00', 54) , ('01.01.2014 13:45:00', 55) , ('01.01.2014 14:00:00', 56) , 
                 ('01.01.2014 14:15:00', 57),('01.01.2014 14:30:00', 58),('01.01.2014 14:45:00', 59),
                 ('01.01.2014 15:00:00', 60),('01.01.2014 15:15:00', 61),('01.01.2014 15:30:00', 62),
                 ('01.01.2014 15:45:00', 63),('01.01.2014 16:00:00', 64),('01.01.2014 16:15:00', 65) , 
                 ('01.01.2014 16:30:00', 66),('01.01.2014 16:45:00', 67),('01.01.2014 17:00:00', 68),
                 ('01.01.2014 17:15:00', 69),('01.01.2014 17:30:00', 70),('01.01.2014 17:45:00', 71),
                 ('01.01.2014 18:00:00', 72),('01.01.2014 18:15:00', 73),('01.01.2014 18:30:00', 74) , 
                 ('01.01.2014 18:45:00', 75) , ('01.01.2014 19:00:00', 76) , ('01.01.2014 19:15:00', 77),
                 ('01.01.2014 19:30:00', 78),('01.01.2014 19:45:00', 79),('01.01.2014 20:00:00', 80),
                 ('01.01.2014 20:15:00', 81),('01.01.2014 20:30:00', 82),('01.01.2014 20:45:00', 83),
                 ('01.01.2014 21:00:00',84),('01.01.2014 21:15:00', 85) , ('01.01.2014 21:30:00', 86),
                 ('01.01.2014 21:45:00', 87),('01.01.2014 22:00:00', 88),('01.01.2014 22:15:00', 89),
                 ('01.01.2014 22:30:00', 90),('01.01.2014 22:45:00', 91),('01.01.2014 23:00:00', 92),
                 ('01.01.2014 23:15:00', 93),('01.01.2014 23:30:00', 94) , ('01.01.2014 23:45:00', 95)])
def ev1arrdeptime():
    with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv') as f:
        reader = csv.reader(f)
        header = next(reader)
        y = []
        for row in reader :
            ydist = float(row[0])
            yaDate = row[1]
            ydDate = row[2]
            y.append([ydist,yaDate,ydDate])
        ev1 =(random.choice(y))
    return(ev1);
def parameterdy( i, T, IE,RE,TS,TD):
    M= i
    STime = [0 for t in range(len(TS))]
    DTime = [0 for t in range(len(TD))]
    for i in range(0,len(TS)):
        STime[i] = TS[i]
        DTime[i] = TD[i]
    path1 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/HH_profile_Scheduling.csv"
    file = open(path1, newline='')
    reader = csv.reader(file)
    header = next(reader)
    HH = []
    for row in reader :
    # row = ['P1[kW]', ' P2[kW]', ' P3[kW]', ' Q1[kVAr]', ' Q2[kVAr]', ' Q3[kVAr]']
        HHDate = row[0]
        HHP_a = float(row[1])
        HHP_b = float(row[2])
        HHP_c = float(row[3])
        HHQ_a = float(row[4])
        HHQ_b = float(row[5])
        HHQ_c = float(row[6])
        HH.append([HHDate,HHP_a,HHP_b,HHP_c,HHQ_a,HHQ_b,HHQ_c])
    path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
    file2 = open(path2, newline='')
    reader2 = csv.reader(file2)
    header2 = next(reader2)
    marketcost = []
    for row in reader2 :
    # row = ['Price [ct/kWh]']
        mcost = float(row[0])
        marketcost.append(mcost)
    cost = [j/100 for j in marketcost]
    path3 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/pv_profile.csv"
    file3 = open(path3, newline='')
    reader3 = csv.reader(file3)
    header3 = next(reader3)
    PV =[]
    for row in reader3 :
    # row = ['Power [kW]']
        PV_P = float(row[0])
        PV.append(PV_P)
    m = Model("dynamicscheduling")
    pa =[[0 for t in range(T)] for i in range(M)]
    pb =[[0 for t in range(T)] for i in range(M)]
    pc =[[0 for t in range(T)] for i in range(M)]
    patot =[0 for t in range(T)]
    pbtot =[0 for t in range(T)]
    pctot =[0 for t in range(T)]
    pahhpv =[0 for t in range(T)]
    pbhhpv =[0 for t in range(T)]
    pchhpv =[0 for t in range(T)]
    pab =[0 for t in range(T)]
    pbc =[0 for t in range(T)]
    pca =[0 for t in range(T)]
    mc =[[0 for t in range(T)] for i in range(M)]
    e =[[0 for t in range(T)] for i in range(M)]
    subcost =[0 for i in range(M)]
    Im_cost =[0 for i in range(M)]
    te =[0 for t in range(T)]
    l= list([(i,t) for i in range(M) for t in range(T)])
    a = m.addVars(l, vtype=GRB.BINARY,name="a")
    b = m.addVars(l, vtype=GRB.BINARY,name="b")
    c = m.addVars(l, vtype=GRB.BINARY,name="c")
    p = m.addVars(l, lb = -7.2, ub = 7.2, name="Power")
    #Phase constraint
    for i in range(M):
        for t in range(T):
            m.addConstr(a[i,t] + b[i,t] + c[i,t] == 1, name="phase_constraint[%d,%d]" %(i,t))
            
    #SOC user constraint
    for i in range(M):    
        for t in range(STime[i],DTime[i]):
            if p[i,t] >= 0:
                e[i][t] +=  p[i,t] * 0.9 * 1/4  # Energy consumed by EV1   1/4 *
            else:
                e[i][t] +=  p[i,t] * 1.1 * 1/4
            mc[i][t] = e[i][t] * cost[t]  # Cost for each EV
            subcost[i] += mc[i][t]
            te[i] += e[i][t]
            n = te[i]
        initial = IE[i] 
        required = RE 
        I_E = int(np.round(initial))
        R_E = int(np.round(required))
        m.addConstr(n >=0,name="SOC_minconstraint[%d]" %i)
        m.addConstr(n <=24,name="SOC_maxconstraint[%d]" %i)
        m.addConstr(n + I_E == R_E, name="User_constraint[%d]" %i)
    
    #power outside TS TD as 0
    for i in range(M):
        for t in range(0,STime[i]) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))
        for t in range(DTime[i],T) :
            m.addConstr(p[i,t] == 0, "Power_constraint[%d,%d]" %(i,t))

    #Grid constraint
    for t in range(T):
        for i in range(M):
    #if i < M :
            pa[i][t] = a[i,t] * p[i,t]
            pb[i][t] = b[i,t] * p[i,t]
            pc[i][t] = c[i,t] * p[i,t]
            
            patot[t] += pa[i][t]
            pbtot[t] += pb[i][t]
            pctot[t] += pc[i][t]
        #print(pa[i][t])
        pahhpv[t] = patot[t] + HH[t][1] - PV[t]
        pbhhpv[t] = pbtot[t] + HH[t][2]
        pchhpv[t] = pctot[t] + HH[t][3]
   
        
        #pab[t] = patot[t] - pbtot[t]
        #pbc[t] = pbtot[t] - pctot[t]
        #pca[t] = pctot[t] - patot[t]
    
        pab[t] = pahhpv[t] - pbhhpv[t]
        pbc[t] = pbhhpv[t] - pchhpv[t]
        pca[t] = pchhpv[t] - pahhpv[t]
    
        m.addConstr(pab[t] <= 4.60, name="Power_A_B_maxconstraint")
        m.addConstr(pab[t] >= -4.60, name="Power_A_B_minconstraint")
        m.addConstr(pbc[t] <= 4.60, name="Power_B_C_maxconstraint")
        m.addConstr(pbc[t] >= -4.60, name="Power_B_C_minconstraint")
        m.addConstr(pca[t] <= 4.60, name="Power_C_A_maxconstraint")
        m.addConstr(pca[t] >= -4.60, name="Power_C_A_minconstraint")
    
    #Total cost:
    C = sum(subcost[i] for i in range(M))
    Im_cost = C
    ST = STime[0]
    i = M - 1
    DT = DTime[i]
    for t in range(ST,DT):
        if pab[t] <= 2.3:
            Im_cost += 0.0
        elif pab[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pbc[t] <= 2.3:
            Im_cost += 0.0
        elif pbc[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
        if pca[t] <= 2.3:
            Im_cost += 0.0
        elif pca[t] >=-2.3:
            Im_cost += 0.0
        else:
            Im_cost += 1.0
    #set objective:
    m.ModelSense = GRB.MINIMIZE
    Im_C = Im_cost
    #m.setObjective(C)
    m.setObjective(Im_C)

    m.write('testDynamicscheduling.lp')
    #m.write('testcost.lp')

    # Compute optimal solution
    m.optimize()
    m.write('testDynamicscheduling.sol')
    #print solution 

    for v in m.getVars():
        print(v.varName, v.x)
        
    print('Obj:', m.objVal)
    finalcost = m.objVal
         
    return(finalcost);
M = 2
T = 96
#power=[0 for i in range(T)]
InitialEgy = [0 for i in range(M)]
energycost1 = [0 for i in range(M)]
IEgy = [0 for i in range(M)] 
ev_a=[0 for i in range(M)]
ev_d=[0 for i in range(M)]
Ev = [0 for i in range(M)]
IE = [0 for i in range(M)]
RE = 17.0
Ev[0] = ev1arrdeptime()
Ev[1] = ev1arrdeptime()
Date = '01.01.2014 '
Time = ':00'
Starttime =[0 for i in range(M)]
Departtime =[0 for i in range(M)]
for i in range(0,M):
    Starttime[i] = Date + Ev[i][1] +Time
    Departtime[i] = Date + Ev[i][2] +Time
    print('energy:',Ev[i][0])
    IE[i] = RE - Ev[i][0]
    print('Initial energy:',IE[i])
    print('ev_a:',Starttime[i])
    print('ev_d:',Departtime[i])
    for item,val in thisdict.items():
        if item == Starttime[i]:
            ev_a[i] = val
        elif item == Departtime[i]:
            ev_d[i] = val
if ev_a[0] < ev_a[1]:        
    TS = [ev_a[0],ev_a[1]]  
    TD= [ev_d[0],ev_d[1]]
    IE =[IE[0],IE[1]]
    print('ev1 arrives before')
    print(TS)
    print(TD)
else:
    TS = [ev_a[1],ev_a[0]]   
    TD= [ev_d[1],ev_d[0]]
    IE =[IE[1],IE[0]]
    print('ev2 arrives before')
    print(TS)
    print(TD)
if TD[0] > TS[1] and TD[0] < 96 and TD[1] > TS[1] and TD[1] < 96:
    if TD[0] < TD[1] or TD[1] < TD[0] or TD[1] == TD[0]:
        print('case 1: Both EVs depart before mid-night')
        Start = [TS[0]]
        Depart = [TD[0]]
        cost1=parameterdy(1,96,IE,RE,Start,Depart)

        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)            
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling1ev.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
            
        with open('testDynamicscheduling1ev.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            power =[]
            p = []
            for row in csv_reader:
                power_str = row[0]
                power_P = float(row[1])
                power.append([power_str,power_P])
            for i in range(len(power)):
                for j in range(TS[0],TS[1]):
                    if power[i][0] == 'Power[%d,%d]' %(0,j):
                        p.append(power[i][1])
            print(p)
            energy =[]
            pwr=[0 for i in range(T)]
            for i in range(len(p)):
                if p[i] >= 0:
                    pwr[i] =  p[i] * 0.9 * 1/4  # Energy consumed by EV1   1/4 *
                else:
                    pwr[i] =  p[i] * 1.1 * 1/4   
                energy.append(pwr[i])
                InitialEgy[0] += energy[i]
            print(energy)
            print(InitialEgy[0])
            path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
            file2 = open(path2, newline='')
            reader2 = csv.reader(file2)
            header2 = next(reader2)
            marketcost = []
            for row in reader2 :
                mcost = float(row[0])
                marketcost.append(mcost)
            cost = [j/100 for j in marketcost]
            i = 0
            for t in range(TS[0],TS[1]):
                energycost1[0] += energy[i] * cost[t]
                i +=1
            print(energycost1[0])            
        Start = [TS[1],TS[1]]
        Depart = [TD[0],TD[1]]
        IE[0] += InitialEgy[0]
        cost2=parameterdy(2,96,IE,RE,Start,Depart)
        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling2evs.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
    
        f.close()
        myfile.close()
        finalcost = energycost1[0] + cost2
        print('final optimal cost:',finalcost)
    
elif TD[0] < TS[0] and TD[0] >= 0 and TD[1] < TS[0] and TD[1] >= 0:
    if TD[0] < TD[1] or TD[1] < TD[0]:
        print('case2: Both EVs depart in the morning')
        Start = [TS[0]]
        Depart = [96]
        cost1=parameterdy(1,96,IE,RE,Start,Depart)

        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)            
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling1ev.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
            
        with open('testDynamicscheduling1ev.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            power =[]
            p = []
            for row in csv_reader:
                power_str = row[0]
                power_P = float(row[1])
                power.append([power_str,power_P])
            for i in range(len(power)):
                for j in range(TS[0],TS[1]):
                    if power[i][0] == 'Power[%d,%d]' %(0,j):
                        p.append(power[i][1])
            print(p)
            energy =[]
            pwr=[0 for i in range(T)]
            for i in range(len(p)):
                if p[i] >= 0:
                    pwr[i] =  p[i] * 0.9 * 1/4  # Energy consumed by EV1   1/4 *
                else:
                    pwr[i] =  p[i] * 1.1 * 1/4  
                energy.append(pwr[i])
                InitialEgy[0] += energy[i]
            print(energy)
            print(InitialEgy[0])
            path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
            file2 = open(path2, newline='')
            reader2 = csv.reader(file2)
            header2 = next(reader2)
            marketcost = []
            for row in reader2 :
                mcost = float(row[0])
                marketcost.append(mcost)
            cost = [j/100 for j in marketcost]    
            i = 0
            for t in range(TS[0],TS[1]):
                energycost1[0] += energy[i] * cost[t]
                i += 1
            print(energycost1[0])            
        Start = [TS[1],TS[1]]
        Depart = [96,96]
        IE[0] += InitialEgy[0]
        cost2=parameterdy(2,96,IE,RE,Start,Depart)
        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling2evs.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
        with open('testDynamicscheduling2evs.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            powr =[]
            po = []
            for row in csv_reader:
                powr_str = row[0]
                powr_P = float(row[1])
                powr.append([powr_str,powr_P])
            for i in range(len(powr)):
                for j in range(TS[1],96):
                    if powr[i][0] == 'Power[%d,%d]' %(0,j) or powr[i][0] == 'Power[%d,%d]' %(1,j):
                        po.append(powr[i][1])
            print(po)
            energy =[]
            pwrs=[0 for i in range(T)]
            for i in range(len(po)):
                if po[i] >= 0:
                    pwrs[i] =  po[i] * 0.9 * 1/4  # Energy consumed by EV1   1/4 *
                else:
                    pwrs[i] =  po[i] * 1.1 * 1/4 
                energy.append(pwrs[i])
                IEgy[0] += energy[i]
            print(energy)
            print(IEgy[0])           
        Start = [0,0]
        Depart = [TD[0],TD[1]]
        IE[0] += IEgy[0]
        cost3=parameterdy(2,96,IE,RE,Start,Depart) 
        with open('testDynamicscheduling.sol', newline='') as f:
            reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
            next(reader)
            next(reader)
            lines = list(reader)

        with open('testDynamicscheduling2evsfinal.csv', 'w', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerows(lines)
    
        f.close()
        myfile.close()
        finalcost = energycost1[0] + cost2 + cost3
        print('final optimal cost:',finalcost)
elif TD[0] > TS[1] and TD[0] < 96 and TD[1] < TS[0] and TD[1] >= 0:
    print('case3: EV1 depart before mid-night and EV2 departs in the morning')
    Start = [TS[0]]
    Depart = [TD[0]]
    cost1=parameterdy(1,96,IE,RE,Start,Depart)

    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)            
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling1ev.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
            
    with open('testDynamicscheduling1ev.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        power =[]
        p = []
        for row in csv_reader:
            power_str = row[0]
            power_P = float(row[1])
            power.append([power_str,power_P])
        for i in range(len(power)):
            for j in range(TS[0],TS[1]):
                if power[i][0] == 'Power[%d,%d]' %(0,j):
                    p.append(power[i][1])
        print(p)
        energy =[]
        pwr=[0 for i in range(T)]
        for i in range(len(p)):
            if p[i] >= 0:
                pwr[i] =  p[i] * 0.9 * 1/4  # Energy consumed by EV1   1/4 *
            else:
                pwr[i] =  p[i] * 1.1 * 1/4 
            energy.append(pwr[i])
            InitialEgy[0] += energy[i]
        print(energy)
        print(InitialEgy[0])
        path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
        file2 = open(path2, newline='')
        reader2 = csv.reader(file2)
        header2 = next(reader2)
        marketcost = []
        for row in reader2 :
            mcost = float(row[0])
            marketcost.append(mcost)
        cost = [j/100 for j in marketcost]    
        i = 0
        for t in range(TS[0],TS[1]):
            energycost1[0] += energy[i] * cost[t]
            i += 1
        print(energycost1[0])            
    Start = [TS[1],TS[1]]
    Depart = [TD[0],96]
    IE[0] += InitialEgy[0]
    cost2=parameterdy(2,96,IE,RE,Start,Depart)
    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling2evs.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    with open('testDynamicscheduling2evs.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        powr =[]
        po = []
        for row in csv_reader:
            powr_str = row[0]
            powr_P = float(row[1])
            powr.append([powr_str,powr_P])
        for i in range(len(powr)):
            for j in range(TS[1],96):
                if powr[i][0] == 'Power[%d,%d]' %(1,j):
                    po.append(powr[i][1])
        print(po)
        energy =[]
        pwrs=[0 for i in range(T)]
        for i in range(len(po)):
            if po[i] >= 0:
                pwrs[i] =  po[i] * 0.9 * 1/4  # Energy consumed by EV1   1/4 *
            else:
                pwrs[i] =  po[i] * 1.1 * 1/4 
            energy.append(pwrs[i])
            IEgy[0] += energy[i]
        print(energy)
        print(IEgy[0])           
    Start = [0]
    Depart = [TD[1]]
    IE[0] += IEgy[0]
    cost3=parameterdy(1,96,IE,RE,Start,Depart) 
    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling2evsfinal.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
    finalcost = energycost1[0] + cost2 + cost3
    print('final optimal cost:',finalcost)       
elif TD[1] > TS[1] and TD[1] < 96 and TD[0] < TS[0] and TD[0] >= 0:
    print('case4: EV2 depart before mid-night and EV1 departs in the morning')
    Start = [TS[0]]
    Depart = [96]
    cost1=parameterdy(1,96,IE,RE,Start,Depart)

    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)            
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling1ev.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
            
    with open('testDynamicscheduling1ev.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        power =[]
        p = []
        for row in csv_reader:
            power_str = row[0]
            power_P = float(row[1])
            power.append([power_str,power_P])
        for i in range(len(power)):
            for j in range(TS[0],TS[1]):
                if power[i][0] == 'Power[%d,%d]' %(0,j):
                    p.append(power[i][1])
        print(p)
        energy =[]
        pwr=[0 for i in range(T)]
        for i in range(len(p)):
            if p[i] >= 0:
                pwr[i] =  p[i] * 0.9 * 1/4  # Energy consumed by EV1   1/4 *
            else:
                pwr[i] =  p[i] * 1.1 * 1/4             
            energy.append(pwr[i])
            InitialEgy[0] += energy[i]
        print(energy)
        print(InitialEgy[0])
        path2 = "C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/dynamic_market_price.csv"
        file2 = open(path2, newline='')
        reader2 = csv.reader(file2)
        header2 = next(reader2)
        marketcost = []
        for row in reader2 :
            mcost = float(row[0])
            marketcost.append(mcost)
        cost = [j/100 for j in marketcost]    
        i = 0
        for t in range(TS[0],TS[1]):
            energycost1[0] += energy[i] * cost[t]
            i += 1
        print(energycost1[0])            
    Start = [TS[1],TS[1]]
    Depart = [96,TD[1]]
    IE[0] += InitialEgy[0]
    cost2=parameterdy(2,96,IE,RE,Start,Depart)
    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling2evs.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    with open('testDynamicscheduling2evs.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        powr =[]
        po = []
        for row in csv_reader:
            powr_str = row[0]
            powr_P = float(row[1])
            powr.append([powr_str,powr_P])
        for i in range(len(powr)):
            for j in range(TS[1],96):
                if powr[i][0] == 'Power[%d,%d]' %(0,j):
                    po.append(powr[i][1])
        print(po)
        energy =[]
        pwrs=[0 for i in range(T)]
        for i in range(len(po)):
            if po[i] >= 0:
                pwrs[i] =  po[i] * 0.9 * 1/4  # Energy consumed by EV1   1/4 *
            else:
                pwrs[i] =  po[i] * 1.1 * 1/4 
            energy.append(pwrs[i])
            IEgy[0] += energy[i]
        print(energy)
        print(IEgy[0])           
    Start = [0]
    Depart = [TD[0]]
    IE[0] += IEgy[0]
    cost3=parameterdy(1,96,IE,RE,Start,Depart) 
    with open('testDynamicscheduling.sol', newline='') as f:
        reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
        next(reader)
        next(reader)
        lines = list(reader)

    with open('testDynamicscheduling2evsfinal.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',')
        wr.writerows(lines)
    
    f.close()
    myfile.close()
    finalcost = energycost1[0] + cost2 + cost3
    print('final optimal cost:',finalcost)  
else:
    exit

energy: 11.56
Initial energy: 5.4399999999999995
ev_a: 01.01.2014 17:45:00
ev_d: 01.01.2014 09:00:00
energy: 6.528
Initial energy: 10.472000000000001
ev_a: 01.01.2014 15:30:00
ev_d: 01.01.2014 23:45:00
ev2 arrives before
[62, 71]
[95, 36]
case3: EV1 depart before mid-night and EV2 departs in the morning
Optimize a model with 162 rows, 384 columns and 450 nonzeros
Model has 576 quadratic constraints
Variable types: 96 continuous, 288 integer (288 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [7e-02, 7e-02]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 2e+01]
  QRHS range       [3e-01, 9e+00]
Presolve removed 128 rows and 252 columns
Presolve time: 0.02s
Presolved: 826 rows, 528 columns, 2310 nonzeros
Variable types: 429 continuous, 99 integer (99 binary)

Root relaxation: objective 3.432781e+00, 312 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Ex

Optimize a model with 341 rows, 768 columns and 866 nonzeros
Model has 576 quadratic constraints
Variable types: 192 continuous, 576 integer (576 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [7e-02, 7e-02]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 2e+01]
  QRHS range       [3e-01, 9e+00]
Presolve removed 290 rows and 572 columns
Presolve time: 0.02s
Presolved: 1083 rows, 784 columns, 3430 nonzeros
Variable types: 637 continuous, 147 integer (147 binary)

Root relaxation: objective 2.312248e+00, 421 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.31225    0   62          -    2.31225      -     -    0s
H    0     0                       2.5767796    2.31225  10.3%     -    0s
H    0     0                       2.5584970    2.31225  9.62%     -    0s
H

 514203 302794    2.45508   72   32    2.45676    2.42801  1.17%  16.9  255s
 526149 310592    2.44004   51   61    2.45676    2.42810  1.17%  16.9  260s
 535702 316732    2.45529   72   26    2.45676    2.42815  1.16%  16.9  265s
 547320 324258    2.43070   54   69    2.45676    2.42821  1.16%  16.9  270s
 559001 331882    2.45534   71   29    2.45676    2.42828  1.16%  16.9  275s
 568390 338020    2.43873   51   67    2.45676    2.42832  1.16%  16.9  280s
 581771 346621    2.44311   48   59    2.45676    2.42839  1.15%  16.9  285s
 593672 354378    2.43766   50   63    2.45676    2.42844  1.15%  16.9  290s
 605211 361778    2.45164   70   36    2.45676    2.42851  1.15%  16.9  295s
 616400 368827    2.44270   53   52    2.45676    2.42855  1.15%  16.9  300s
 628402 376551    2.45411   58   52    2.45676    2.42861  1.15%  16.9  305s
 639917 383881    2.44026   54   57    2.45676    2.42866  1.14%  16.9  310s
 651449 391295    2.42920   46   74    2.45676    2.42871  1.14%  16.9  315s

 1557595 940009    2.44705   60   46    2.45676    2.43114  1.04%  17.6  785s
 1565689 944860    2.45660   64   38    2.45676    2.43116  1.04%  17.6  790s
 1573431 949256    2.45635   73   34    2.45676    2.43117  1.04%  17.6  795s
 1581425 953836    2.45295   62   38    2.45676    2.43119  1.04%  17.6  800s
 1589003 958288    2.44521   55   55    2.45676    2.43120  1.04%  17.6  806s
 1595185 961962    2.44828   67   41    2.45676    2.43121  1.04%  17.6  810s
 1602712 966343    2.44425   53   58    2.45676    2.43123  1.04%  17.6  817s
 1604490 967403    2.45494   63   46    2.45676    2.43123  1.04%  17.6  820s
 1611914 971713     cutoff   58         2.45676    2.43125  1.04%  17.6  825s
 1619699 976251    2.44955   48   60    2.45676    2.43127  1.04%  17.6  831s
 1625538 979660    2.44941   54   56    2.45676    2.43127  1.04%  17.6  835s
 1631312 982930    2.43988   56   51    2.45676    2.43129  1.04%  17.6  840s
 1639466 987641    2.43815   48   69    2.45676    2.43130  1.04

 2236492 1333922    2.44711   59   51    2.45676    2.43219  1.00%  17.8 1305s
 2244595 1338571    2.45480   67   32    2.45676    2.43220  1.00%  17.8 1311s
 2250255 1341870    2.44859   66   38    2.45676    2.43221  1.00%  17.8 1316s
 2256178 1345367    2.44630   53   53    2.45676    2.43222  1.00%  17.9 1321s
 2262078 1348796    2.44975   64   45    2.45676    2.43223  1.00%  17.9 1326s
 2267851 1352082    2.45476   57   64    2.45676    2.43224  1.00%  17.9 1330s
 2273780 1355529     cutoff   64         2.45676    2.43224  1.00%  17.9 1335s
 2279873 1359136    2.43371   46   68    2.45676    2.43225  1.00%  17.9 1340s
 2287136 1363353    2.44861   60   56    2.45676    2.43226  1.00%  17.9 1345s
 2292916 1366654    2.45355   56   48    2.45676    2.43226  1.00%  17.9 1350s
 2298828 1370157    2.43855   50   60    2.45676    2.43227  1.00%  17.9 1355s
 2304710 1373523    2.44000   54   66    2.45676    2.43228  1.00%  17.9 1360s
 2312540 1378039    2.44261   53   53    2.45676    

...

 23329337 12387668    2.45076   63   44    2.45676    2.43890  0.73%  18.3 70134s
 23331250 12388591    2.45533   69   26    2.45676    2.43891  0.73%  18.3 70148s
 23333206 12389546    2.45464   67   40    2.45676    2.43891  0.73%  18.3 70163s
 23335293 12390752    2.45425   67   39    2.45676    2.43891  0.73%  18.3 70174s
 23336958 12391548    2.44578   52   50    2.45676    2.43891  0.73%  18.3 70187s
 23338983 12392580    2.45638   62   44    2.45676    2.43891  0.73%  18.3 70200s
 23341010 12393585     cutoff   69         2.45676    2.43891  0.73%  18.3 70215s
 23342910 12394614    2.44876   55   54    2.45676    2.43891  0.73%  18.3 70228s
 23344694 12395544    2.44581   48   58    2.45676    2.43891  0.73%  18.3 70242s
 23346536 12396397    2.45662   73   20    2.45676    2.43891  0.73%  18.3 70256s
 23348135 12397280    2.45258   55   51    2.45676    2.43891  0.73%  18.3 70270s
 23350121 12398287    2.45170   65   42    2.45676    2.43891  0.73%  18.3 70284s
 23352417 123994

 23521282 12484191    2.44324   53   52    2.45676    2.43892  0.73%  18.3 71441s
 23523368 12485309    2.43952   54   50    2.45676    2.43892  0.73%  18.3 71453s
 23525165 12486187    2.44451   45   66    2.45676    2.43893  0.73%  18.3 71465s
 23526890 12487000    2.45035   61   55    2.45676    2.43893  0.73%  18.3 71478s
 23528727 12487921    2.44176   57   50    2.45676    2.43893  0.73%  18.3 71489s
 23530929 12489010    2.44932   62   46    2.45676    2.43893  0.73%  18.3 71502s
 23532897 12489990    2.43947   54   56    2.45676    2.43893  0.73%  18.3 71514s
 23534389 12490683    2.45102   61   41    2.45676    2.43893  0.73%  18.3 71526s
 23536175 12491565    2.45534   71   30    2.45676    2.43893  0.73%  18.3 71537s
 23538094 12492503     cutoff   68         2.45676    2.43893  0.73%  18.3 71551s
 23540417 12493637    2.45533   69   30    2.45676    2.43893  0.73%  18.3 71564s
 23542370 12494641     cutoff   74         2.45676    2.43893  0.73%  18.3 71577s
 23544320 124955

 23710835 12579407    2.45467   63   45    2.45676    2.43894  0.73%  18.3 72821s
 23712584 12580258     cutoff   65         2.45676    2.43894  0.73%  18.3 72834s
 23714440 12581228    2.45152   69   36    2.45676    2.43894  0.73%  18.3 72846s
 23716537 12582242     cutoff   65         2.45676    2.43894  0.73%  18.3 72860s
 23718572 12583147    2.45407   62   45    2.45676    2.43894  0.73%  18.3 72873s
 23720422 12584130    2.44333   51   54    2.45676    2.43895  0.73%  18.3 72887s
 23722389 12585096    2.44786   60   49    2.45676    2.43895  0.73%  18.3 72901s
 23724312 12586065    2.45656   76   24    2.45676    2.43895  0.73%  18.3 72916s
 23725790 12586768    2.44652   56   53    2.45676    2.43895  0.73%  18.3 72930s
 23728166 12588012    2.43899   52   70    2.45676    2.43895  0.73%  18.3 72942s
 23730079 12588918    2.45634   68   33    2.45676    2.43895  0.73%  18.3 72956s
 23731979 12589827    2.45088   57   44    2.45676    2.43895  0.73%  18.3 72969s
 23734077 125909

 23900259 12674080    2.45331   67   31    2.45676    2.43896  0.72%  18.3 74200s
 23902169 12675006    2.45329   67   39    2.45676    2.43896  0.72%  18.3 74214s
 23904344 12676097    2.45580   73   31    2.45676    2.43896  0.72%  18.3 74228s
 23906170 12677066    2.45671   73   16    2.45676    2.43896  0.72%  18.3 74243s
 23908217 12677994    2.44715   60   53    2.45676    2.43896  0.72%  18.3 74256s
 23910106 12678918    2.44336   47   69    2.45676    2.43896  0.72%  18.3 74269s
 23911956 12679883    2.45508   70   31    2.45676    2.43896  0.72%  18.3 74284s
 23913691 12680746    2.45644   71   24    2.45676    2.43896  0.72%  18.3 74297s
 23915871 12681808     cutoff   79         2.45676    2.43896  0.72%  18.3 74311s
 23918046 12682796    2.45443   62   44    2.45676    2.43896  0.72%  18.3 74324s
 23919869 12683718     cutoff   69         2.45676    2.43896  0.72%  18.3 74338s
 23921638 12684591    2.45099   56   58    2.45676    2.43896  0.72%  18.3 74353s
 23923089 126853

 24092122 12770176    2.44339   55   55    2.45676    2.43898  0.72%  18.3 75543s
 24094006 12771077    2.44695   54   56    2.45676    2.43898  0.72%  18.3 75558s
 24095833 12771993    2.44848   61   38    2.45676    2.43898  0.72%  18.3 75569s
 24097738 12772948    2.45614   64   39    2.45676    2.43898  0.72%  18.3 75580s
 24099622 12773837     cutoff   74         2.45676    2.43898  0.72%  18.3 75591s
 24101808 12774862    2.44184   60   52    2.45676    2.43898  0.72%  18.3 75602s
 24103523 12775722    2.44804   62   46    2.45676    2.43898  0.72%  18.3 75613s
 24105452 12776762    2.43949   57   57    2.45676    2.43898  0.72%  18.3 75628s
 24107376 12777643    2.45321   65   37    2.45676    2.43898  0.72%  18.3 75641s
 24109284 12778650     cutoff   70         2.45676    2.43898  0.72%  18.3 75654s
 24111157 12779572    2.44549   57   54    2.45676    2.43898  0.72%  18.3 75668s
 24113264 12780705    2.44849   57   48    2.45676    2.43898  0.72%  18.3 75681s
 24114994 127815

In [ ]:
#Sample set
import random
import csv
dicti = {
  '00:00': 0,
  '00:15': 1,
  '00:30': 2
}
dicti.update([ ('00:45', 3) , ('01:00', 4) , ('01:15', 5) ,
                 ('01:30', 6),('01:45', 7),('02:00', 8),
                 ('02:15', 9),('02:30', 10),('02:45', 11),
                 ('03:00', 12),('03:15', 13),('03:30', 14) , 
                 ('03:45', 15) , ('04:00', 16) , ('04:15', 17),
                 ('04:30', 18),('04:45', 19),('05:00', 20),
                 ('05:15', 21),('05:30', 22),('05:45', 23),
                 ('06:00', 24),('06:15', 25) , ('06:30', 26),
                 ('06:45', 27),('07:00', 28),('07:15', 29),
                 ('07:30', 30),('07:45', 31),('08:00', 32),
                 ('08:15', 33),('08:30', 34) , ('08:45', 35) , 
                 ('09:00', 36) , ('09:15', 37),('09:30', 38),
                 ('09:45', 39),('10:00', 40),('10:15', 41),
                 ('10:30', 42),('10:45', 43),('11:00', 44),
                 ('11:15', 45) , ('11:30', 46),('11:45', 47),
                 ('12:00', 48),('12:15', 49),('12:30', 50),
                 ('12:45', 51),('13:00', 52),('13:15', 53),
                 ('13:30', 54) , ('13:45', 55) , ('14:00', 56) , 
                 ('14:15', 57),('14:30', 58),('14:45', 59),
                 ('15:00', 60),('15:15', 61),('15:30', 62),
                 ('15:45', 63),('16:00', 64),('16:15', 65) , 
                 ('16:30', 66),('16:45', 67),('17:00', 68),
                 ('17:15', 69),('17:30', 70),('17:45', 71),
                 ('18:00', 72),('18:15', 73),('18:30', 74) , 
                 ('18:45', 75) , ('19:00', 76) , ('19:15', 77),
                 ('19:30', 78),('19:45', 79),('20:00', 80),
                 ('20:15', 81),('20:30', 82),('20:45', 83),
                 ('21:00',84),('21:15', 85) , ('21:30', 86),
                 ('21:45', 87),('22:00', 88),('22:15', 89),
                 ('22:30', 90),('22:45', 91),('23:00', 92),
                 ('23:15', 93),('23:30', 94) , ('23:45', 95)])
with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/Arr_Dep_EV.csv', newline='') as f:
    reader = csv.reader(f)
    #reader = csv.reader((line.replace('  ', ' ') for line in f), delimiter=' ')
    header = next(reader)
    y = []
    for row in reader :
            ydist = row[0]
            yaDate = row[1]
            ydDate = row[2]
            y.append([ydist,yaDate,ydDate])

    ev1 =(random.choice(y))
    ev2 =(random.choice(y))
    print(ev1)
    print(ev2)
    for item,val in dicti.items():
        if item == ev1[1]:
            ev1_a = val
        elif item == ev1[2]:
            ev1_d = val
        elif item == ev2[1]:
            ev2_a = val
        elif item == ev2[2]:
            ev2_d = val
    if ev1_a < ev2_a and ev2_a < ev1_d and ev1_d < ev2_d:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
        #print('ev2 between ev1a and ev1d')
    elif ev2_a < ev1_a and ev1_a < ev2_d and ev2_d < ev1_d:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
        #print('ev1 between ev2a and ev2d')
    elif ev1_a < ev2_a and ev2_a < ev2_d and ev1_d < ev1_a:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
        #print('ev2 between ev1a and ev1d')
    elif ev2_a < ev1_a and ev1_a < ev1_d and ev2_d < ev2_a:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
    elif ev1_d < ev2_d and ev2_d < ev1_a and ev1_a < ev2_a:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
        #print('ev2 between ev1a and ev1d')
    elif ev2_d < ev1_d and ev1_d < ev2_a and ev2_a < ev1_a:
        with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, delimiter=',')
            wr.writerow(ev1)
            wr.writerow(ev2)
#Distance to energy
import csv
from scipy import interpolate
distance =[]
with open('C:/Users/Subha Thillairajan/Documents/thesis docs/thesis csv files/EVData.csv') as file:
    reader = csv.reader(file)
    #header = next(reader)
    x = []
    for row in reader :
        xdist = float(row[0])
        xaDate = row[1]
        xdDate = row[2]
        x.append([xdist,xaDate,xdDate])
    #ev2 =(random.choice(x)) 
    for i in range(len(x)):
        #print(x[i][0])
        distance.append(x[i][0])
print(distance)
Distance = [0,100]    #Distance given /100 km
Energy = [0,17]  #Energy in kwh
f = interpolate.interp1d(Distance,Energy)
for i in range(len(distance)):
    evdistance = distance[i] 
    Energyeq = f(evdistance)
    print('Energy equivalent:',Energyeq)